In [1]:
# Erasmus+ ICCT project (2018-1-SI01-KA203-047081)

# Toggle cell visibility

from IPython.display import HTML
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.input').hide()
    } else {
        $('div.input').show()
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
Toggle cell visibility <a href="javascript:code_toggle()">here</a>.''')
display(tag)

# Hide the code completely

# from IPython.display import HTML
# tag = HTML('''<style>
# div.input {
#     display:none;
# }
# </style>''')
# display(tag)


## Controllabilità

La **controllabilità** è una proprietà fondamentale di un sistema che descrive se lo stato del sistema $x(t)$ può essere portato, con un input appropriato $u(t)$, in un qualche punto dello spazio degli stati oppure no.
Più specificamente, la controllabilità fa capire se, partendo da uno stato specifico e variando gli input, è possibile o meno portare il sistema nell'origine dello spazio degli stati. Se è possibile, si dice che quello stato è controllabile (a zero).
L'analoga proprietà di **Raggiungibilità** indica se partendo dall'origine è possibile raggiungere uno stato specifico.
Se è possibile, si dice che quello stato è raggiungibile (da zero).
Per i sistemi lineari tempo invarianti a tempo continuo, l'insieme degli stati raggiungibili e quello degli stati controllabili coincidono, si tratta di spazi vettoriali, e si può parlare di controllabilità e raggiungibilità del sistema in generale.

Dato un sistema lineare del tipo:

\begin{cases}
\dot{\textbf{x}}=A\textbf{x}+B\textbf{u} \\ 
\textbf{y}=C\textbf{x}
\end{cases}

e definendo l'insieme raggiungibile come $R=\{x\, |\, \exists\, u\in\, U: x=\int_{t_0}^{t_f}e^{A(t_f-\tau)}Bu(\tau)d\tau\}$, un sistema è raggiungibile se e solo se $\text{dim}(R)=n$ dove $n:x\in \mathbb{R}^n$. Si può dimostrare che $R$ è uno spazio vettoriale e che è descritto dal _gramiano di raggiungibilità_ $G_r(0,t_f)=\int_{0}^{t_f}e^{A(t_f-\tau)}BB^Te^{A^T(t_f-\tau)}d\tau$; quindi, un sistema è controllabile se e solo se $\text{det}(G_r)\neq 0$ e introducendo la _matrice di controllabilità_
$$
\mathcal{C} = \begin{bmatrix}B&AB&A^2B&\dots&A^{n-1}B\end{bmatrix}
$$
se e solo se $\mathcal{C}$ ha rango massimo per colonne poiché l'immagine di $G_r$ è la stessa di $\mathcal{C}$.

La proprietà di controllabilità è fondamentale perché se un punto dello spazio degli stati non è controllabile / raggiungibile allora non esiste una funzione di input $u(t)$ che porti lo stato del sistema a questo punto. Ciò a sua volta implica che non sarà nemmeno possibile creare un controllore ad anello chiuso che regoli lo stato del sistema fino a quel punto.

L'esempio seguente consente di modificare le matrici $A$ e $B$, calcolare la corrispondente matrice di controllabilità $\mathcal{C}$ e tracciare la risposta a vari input e condizioni iniziali.

### Come utilizzare questo notebook?
- Prova a rendere incontrollabile il sistema predefinito modificando solo $B$.
- Prova a definire una coppia $(A,B)$ che abbia un autovalore positivo associato a uno stato incontrollabile. Come si comporta l'output in una risposta al gradino?

In [2]:
#Preparatory Cell 

%matplotlib inline
import control
import numpy
from IPython.display import display, Markdown, Javascript
import ipywidgets as widgets
import matplotlib.pyplot as plt

#print a matrix latex-like
def bmatrix(a):
     """Returns a LaTeX bmatrix - by Damir Arbula (ICCT project)

     :a: numpy array
     :returns: LaTeX bmatrix as a string
     """
     if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
     lines = str(a).replace('[', '').replace(']', '').splitlines()
     rv = [r'\begin{bmatrix}']
     rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
     rv +=  [r'\end{bmatrix}']
     return ''.join(rv) #'\n'.join(rv)

#matrixWidget is a matrix looking widget built with a VBox of HBox(es) that returns a numPy array as value !
class matrixWidget(widgets.VBox):
    def updateM(self,change):
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.M_[irow,icol] = self.children[irow].children[icol].value
                #print(self.M_[irow,icol])
        self.value = self.M_

    def dummychangecallback(self,change):
        pass
    
    
    def __init__(self,n,m):
        self.n = n
        self.m = m
        self.M_ = numpy.matrix(numpy.zeros((self.n,self.m)))
        self.value = self.M_
        widgets.VBox.__init__(self,
                             children = [
                                 widgets.HBox(children = 
                                              [widgets.FloatText(value=0.0, layout=widgets.Layout(width='90px')) for i in range(m)]
                                             ) 
                                 for j in range(n)
                             ])
        
        #fill in widgets and tell interact to call updateM each time a children changes value
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
                self.children[irow].children[icol].observe(self.updateM, names='value')
        #value = Unicode('example@example.com', help="The email value.").tag(sync=True)
        self.observe(self.updateM, names='value', type= 'All')
        
    def setM(self, newM):
        #disable callbacks, change values, and reenable
        self.unobserve(self.updateM, names='value', type= 'All')
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].unobserve(self.updateM, names='value')
        self.M_ = newM
        self.value = self.M_
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].observe(self.updateM, names='value')
        self.observe(self.updateM, names='value', type= 'All')        

                #self.children[irow].children[icol].observe(self.updateM, names='value')

             
#overlaod class for state space systems that DO NOT remove "useless" states (what "professor" of automatic control would do this?)
class sss(control.StateSpace):
    def __init__(self,*args):
        #call base class init constructor
        control.StateSpace.__init__(self,*args)
    #disable function below in base class
    def _remove_useless_states(self):
        pass

In [3]:
# Preparatory cell
A = numpy.matrix('-1 1 0; 0 -1 1; 0 0 -1')
B = numpy.matrix('0; 0; 1')
C = numpy.matrix('1 0 0')
X0 = numpy.matrix('0; 0; 0')

Aw = matrixWidget(3,3)
Aw.setM(A)
Bw = matrixWidget(3,1)
Bw.setM(B)
Cw = matrixWidget(1,3)
Cw.setM(C)
X0w = matrixWidget(3,1)
X0w.setM(X0)

In [4]:
# Misc
#define type of ipout 
selu = widgets.Dropdown(
    options=[('impulso','impulse'), ('gradino','step'), ('sinusoide','sinusoid'), ('onda quadra','square wave')],
    value='step',
    description='Input:',
    disabled=False
)
# Define the values of the input
u = widgets.FloatSlider(
    value=1,
    min=0,
    max=20.0,
    step=0.1,
    description=r'$u$:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
period = widgets.FloatSlider(
    value=1,
    min=0.01,
    max=4,
    step=0.01,
    description='Periodo: ',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

#create dummy widget 
DW = widgets.FloatText(layout=widgets.Layout(width='0px', height='0px'))

#create button widget
START = widgets.Button(
    description='Test',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Test',
    icon='check'
)
                       
def on_start_button_clicked(b):
    #This is a workaround to have intreactive_output call the callback:
    #   force the value of the dummy widget to change
    if DW.value> 0 :
        DW.value = -1
    else: 
        DW.value = 1
    pass
START.on_click(on_start_button_clicked)

In [7]:
def main_callback(A, B, C, X0, u, period, selu, DW):
    eig = numpy.linalg.eig(A)[0]
    R = control.ctrb(A,B)
    rankR = numpy.linalg.matrix_rank(R)
    
    text = r'Gli autovalori del sistema sono ' + str(eig[0]) + r', ' + str(eig[1])+ r' e ' + str(eig[2]) + r'. '
    if rankR == 3:
        text = text + r'Siccome $\text{rank}(\mathcal{C})=3$ il sistema è controllabile. ' + '\n\n' + 'La matrice di raggiungibilità è: ' + r'$\mathcal{C}=' + bmatrix(R) + r'$'
    else:
        text = text + r'Siccome $\text{rank}(\mathcal{C})=' + str(rankR) + r'$ il sistema non è controllabile. ' + '\n\n' + 'La matrice di raggiungibilità è: ' + r'$\mathcal{C}=' + bmatrix(R) + r'$'
    display(Markdown(text))
    
    
    sys = sss(A,B,C,0)
    if numpy.real([eig[0],eig[1],eig[2]])[0] == 0 and numpy.real([eig[0],eig[1],eig[2]])[1] == 0 and numpy.real([eig[0],eig[1],eig[2]])[2] == 0:
        T = numpy.linspace(0,20,1000)
    else:
        if min(numpy.abs(numpy.real([eig[0],eig[1],eig[2]]))) != 0:
            T = numpy.linspace(0,7*1/min(numpy.abs(numpy.real([eig[0],eig[1],eig[2]]))),1000)
        else:
            T = numpy.linspace(0,7*1/max(numpy.abs(numpy.real([eig[0],eig[1],eig[2]]))),1000)
    
    if selu == 'impulse': #selu
        U = [0 for t in range(0,len(T))]
        U[0] = u
        T, Y, X = control.forced_response(sys,T=T,U=U,X0=X0)
    if selu == 'step':
        U = [u for t in range(0,len(T))]
        T, Y, X = control.forced_response(sys,T=T,U=U,X0=X0)
    if selu == 'sinusoid':
        U = u*numpy.sin(2*numpy.pi/period*T)
        T, Y, X = control.forced_response(sys,T=T,U=U,X0=X0)
    if selu == 'square wave':
        U = u*numpy.sign(numpy.sin(2*numpy.pi/period*T))
        T, Y, X = control.forced_response(sys,T=T,U=U,X0=X0)
    
    fig = plt.figure(figsize=(16,5))
    
    fig.add_subplot(121)
    plt.title('Risposta: stati')
    plt.plot(T,X[0])
    plt.plot(T,X[1])
    plt.plot(T,X[2])
    plt.ylabel('Stati')
    plt.xlabel('t [s]')
    plt.legend([r'$x_1$',r'$x_2$',r'$x_3$'])
    plt.grid()
    
    fig.add_subplot(122)
    plt.ylabel(r'$y$')
    plt.plot(T,Y)
    plt.xlabel('t [s]')
    plt.title('Risposta: uscita')
    plt.grid()
    

alltogether = widgets.VBox([widgets.HBox([widgets.Label('A:',border=3),  Aw, widgets.Label('   ',border=3),
                                          widgets.Label('B:',border=3),  Bw, widgets.Label('   ',border=3),
                                          widgets.Label('x0:',border=3),X0w,widgets.Label('   ',border=3),
                                          START]),
                            widgets.Label('   ',border=3),
                            widgets.HBox([widgets.Label('C:',border=3),Cw,widgets.Label('   ',border=3)]),
                            widgets.Label('   ',border=3),
                            widgets.HBox([selu, u, period])])
out = widgets.interactive_output(main_callback, {'A':Aw, 'B':Bw, 'C':Cw, 'X0':X0w, 'u':u, 'period':period, 'selu':selu, 'DW':DW})
out.layout.height = '460px'
display(out, alltogether)

Output(layout=Layout(height='460px'))

In [6]:
%%js
Jupyter.notebook.execute_cells([6])

<IPython.core.display.Javascript object>